In [2]:
import numpy as np
import pandas as pd

In [3]:
# A single node of the tree - either Decision or leaf
class Node():
    def __init__(self, feature_index=None, threshold=None, left_child=None, right_child=None, info_gain = None, value=None):
        
        #only for the decision nodes
        self.feature_index = feature_index
        self.threshold = threshold
        self.left_child = left_child
        self.right_child = right_child
        self.info_gain = info_gain
        
        # only for leaf node
        self.value = value

In [4]:
# A class to build the whole decision tree
class Decision_tree():
    def __init__(self, max_depth=3, min_sample_split=2, gain_fn="gini",n_features=None):
        """
        Params: 
        max_depth - int, the maximum depth of the tree
        min_sample_split - int, the minimum number of splits on the given data
        """
        self.gain_fn = gain_fn
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.n_features = n_features
    
    def optimal_tree(self, data, curr_depth=0):
        # extract the data
        X, y = data[:,:-1], data[:,-1]
        n_samples, n_features = X.shape
        # feature selection if number of features are gven
        self.n_features = n_features if self.n_features is None else np.minimum(self.n_features, n_features)
        
        if n_samples > self.min_sample_split and curr_depth <= self.max_depth:
            # to find the best decision split for the node
            feat_idxs = np.random.choice(n_features, self.n_features, replace=False)
            
            best_decision = self.get_best_decision(data, n_samples, feat_idxs) # A dict
            
            if best_decision["entropy_gain"] > 0:
                # building the left tree of the parent node
                left_subtree = self.optimal_tree(best_decision["dataset_left"], curr_depth+1)
                # building the right tree of the parent node
                right_subtree = self.optimal_tree(best_decision["dataset_right"], curr_depth+1)
                
                # forming the tree using decision nodes
                return Node(best_decision["feature_idx"], best_decision["threshold"], 
                            left_subtree, right_subtree, best_decision["entropy_gain"])
            
        value = self.calculate_leaf_value(y)
            
        # inserting the leaf nodes
        return Node(value=value)
            
            
    def calculate_leaf_value(self, Y):
        Y = list(Y)
        return max(Y, key = Y.count)
    
    def get_best_decision(self, data, n_samples, n_features_idx):
        
            best_decision = {}
            max_entropy_gain = -float("inf")
            
            for feature_idx in n_features_idx:
                feature = data[:,feature_idx]
                
                poss_thres = np.unique(feature)
            
                for tres in poss_thres:
                    data_left, data_right = self.data_split_thresh(data,feature_idx, tres)

                    if len(data_left) > 0 and len(data_right) > 0:
                        y,y_left,y_right = data[:,:-1], data_left[:,:-1], data_right[:,:-1]

                        curr_entropy_gain = self.entropy_gain(y, y_left, y_right,self.gain_fn)

                        if curr_entropy_gain > max_entropy_gain:
                            best_decision["feature_idx"] = feature_idx
                            best_decision["threshold"] = tres
                            best_decision["dataset_left"] = data_left
                            best_decision["dataset_right"] = data_right
                            best_decision["entropy_gain"] = curr_entropy_gain
                            max_entropy_gain = curr_entropy_gain
        
            return best_decision
        
    def data_split_thresh(self,data, feature_idx, tres):
        
        data_left = [row for row in data if row[feature_idx] <= tres]
        data_right = [row for row in data if row[feature_idx] > tres]
        return np.array(data_left), np.array(data_right)
    
    def entropy_gain(self, des_node, l_child, r_child, mode="entropy"):
        
        w_l = len(l_child)/len(des_node)
        w_r = len(r_child)/len(des_node)
        
        if mode == "gini":
            info_gain = self.gini_impurity(des_node) - ((w_l * self.gini_impurity(l_child)) + (w_r * self.gini_impurity(r_child)))
            
        else:
            info_gain = self.entropy(des_node) - ((w_l * self.entropy(l_child)) + (w_r * self.entropy(r_child)))
        return info_gain
    
    def entropy(self, y):
        '''
        A function to find the entropy gain
        '''
        unique_idx = np.unique(y)
        entropy = 0
        for idx in unique_idx:
            p_y_ = len(y[y == idx]) / len(y)
            entropy += -(p_y_)*np.log2(p_y_)
            
        return entropy
    
    def gini_impurity(self, y):
        """
        A function to find the gini impurities
        """
        unique_idx = np.unique(y)
        gini_imp = 1
        for idx in unique_idx:
            p_y_ = len(y[y == idx]) / len(y)
            gini_imp -= (p_y_ ** 2)
            
        return gini_imp
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left_child, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right_child, indent + indent)
    
    def fit(self, X, Y):
        # Training for the optimal fit
        data = np.concatenate((X, Y), axis=1)
        self.root = self.optimal_tree(data)
    
    def predict(self, X):
        # Predicting the testing data
        pred_labels = [int(self.decision_trees_pred(x, self.root)) for x in X]
        
        return pred_labels
    
    def decision_trees_pred(self, X, tree):
        '''Recursive method to predict from the tree'''
        if tree.value != None:
            return tree.value
        
        feature_val = X[tree.feature_index]
        
        if feature_val<= tree.threshold:
            return self.decision_trees_pred(X, tree.left_child)
        
        else:
            return self.decision_trees_pred(X, tree.right_child)